In [ ]:
!pip install markupsafe==2.0.1
!pip install rpy2

import rpy2.robjects as robjects
from rpy2.robjects import Formula, pandas2ri
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages("fixest")
#utils.install_packages("marginaleffects")
fixest = importr("fixest")
#mgef = importr("marginaleffects")


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


(as ‘lib’ is unspecified)







	‘/tmp/RtmpBIsSjp/downloaded_packages’



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import t as tstats
from scipy.stats.mstats import winsorize
import warnings

warnings.filterwarnings("ignore")

In [ ]:
df_au_eid = pd.read_csv(
    r"path/hbcu/hbcu_au_eid.csv"
)

df_eid = pd.read_csv(
    r"path/hbcu/hbcu_eid.csv"
)

df_eid_citepct = pd.read_csv(
    r"path/hbcu/hbcu_citepct.csv"
)

df_sm_map =pd.read_csv(
    r"path/sm_discipline_lower.csv")

df_jif = pd.read_csv(
    r"path/tenure_jif_0610.csv"
)

df_novel = pd.read_csv(
    r"path/hbcu/hbcu_novel.csv"
)

df_cd = pd.read_csv(
    r"path/hbcu/hbcu_cd_pct.csv"
)

df_multiinst = pd.read_csv(r"path/hbcu/hbcu_multiinst_info.csv")
df_person = pd.read_csv(r"path/hbcu/full_hbcu.csv")


In [ ]:
df_au_eid_match = pd.read_csv(
    r"path/hbcu/hbcu_au_eid_match.csv"
)

df_eid_match = pd.read_csv(
    r"path/hbcu/hbcu_eid_match.csv"
)

df_eid_citepct_match = pd.read_csv(
    r"path/hbcu/hbcu_citepct_match.csv"
)

df_jif = pd.read_csv(
    r"path/tenure_jif_0610.csv"
)

df_novel_match = pd.read_csv(
    r"path/hbcu/hbcu_novel_match.csv"
)

df_cd_match = pd.read_csv(
    r"path/hbcu/hbcu_cd_pct_match.csv"
)

In [ ]:
# mark pubs in 1 and 2 stages

t = df_au_eid.query('year >= 2010 and year <= 2020').merge(df_multiinst, on=['PersonId'], how='inner')

def get_stage(row):
  vars = [row['year'], row['year_start1'], row['year_end1'], row['year_start2'], row['year_end2']]
  for i, var in enumerate(vars):
    if isinstance(var, str):
      vars[i] = 0

  year, year_start1, year_end1, year_start2, year_end2 = vars[0], vars[1], vars[2], vars[3], vars[4]
  # 0: before move
  if year <= year_end1 and year >= year_start1:
    return 0
  # 1: after move
  if year <= year_end2 and year >= year_start2:
    return 1
  else:
    return np.nan
  
t['stage'] = t.apply(lambda row: get_stage(row), axis=1)
t


,Unnamed: 0_x,eid,PersonId,auid,year,Authorseq,last_author_seq,publication_type,doi,isAApub,Unnamed: 0_y,univ1,year_start1,year_end1,isHBCU1,univ2,year_start2,year_end2,isHBCU2,MoveType,period1_len,period2_len,stage
0,1,84857554607,886826,38860994900,2011,1,2,cp,10.1109/pimrc.2011.6139871,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
1,2,85083075356,886826,38860994900,2020,1,1,cp,10.1109/ccwc47524.2020.9031127,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
2,4,85035106301,886826,38860994900,2018,2,6,cp,10.1109/comst.2017.2773628,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
3,7,84883418788,886826,38860994900,2013,1,2,ar,10.1109/tsp.2013.2272554,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
4,10,85028758952,886826,38860994900,2017,4,4,ar,10.1109/tvt.2016.2619418,1.0,1563,North Carolina Agricultural and Technical Stat...,2018,2020,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27915,38147,80755185318,244503,7402187840,2011,1,2,ar,NaN,NaN,1475,North Carolina Agricultural and Technical Stat...,2011,2013,1,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0
27916,38401,84862003184,755214,37088014700,2012,1,1,ar,NaN,NaN,257,University of Maryland Eastern Shore,2014,2020,1,NaN,NaN,NaN,NaN,S,7.0,NaN,0.0
27917,38744,84868706388,802399,14069879700,2012,1,3,ar,NaN,NaN,3077,Jackson State University,2017,2020,1,NaN,NaN,NaN,NaN,S,4.0,NaN,0.0
27918,38861,78650232471,750841,6602222874,2010,5,5,ar,NaN,NaN,1827,Florida A&M University,2013,2020,1,NaN,NaN,NaN,NaN,S,8.0,NaN,0.0


In [ ]:
df_match

,Unnamed: 0,PersonId,PersonId_control,firstname,lastname,firstmiddle,first_intial
0,0,29108,6517,ANNEKE,BART,ANNEKE,A
1,1,167279,13756,RUSSELL,BROWN,RUSSELL,R
2,2,207451,18111,BO,CHEN,BO,B
3,3,753167,24094,DIPANKAR,DASGUPTA,DIPANKAR,D
4,4,388038,22492,GENE,COOPERMAN,GENE D,G
...,...,...,...,...,...,...,...
409,409,372751,364979,JOHN,WAGNER,JOHN R,J
410,410,747079,433766,BENJAMIN,WHITE,BENJAMIN L,B
411,411,806924,433767,DANIEL,WHITEHEAD,DANIEL C,D
412,412,482069,482068,KRISTIN,KEW,KRISTIN L,K


In [ ]:
# mark stages for control group
df_match = pd.read_csv(
    r"path/hbcu/hbcu_match_name.csv"
)

# drop duplicates
#df_match = df_match.sample(frac=1, random_state=1).drop_duplicates(subset=['PersonId'])

t_match = df_au_eid_match.query('year >= 2010 and year <= 2020').merge(df_match[['PersonId', 'PersonId_control']], on='PersonId_control').merge(t[['PersonId', 'year_start1', 'year_end1', 'year_start2', 'year_end2', 'MoveType']], on='PersonId').drop_duplicates()

t_match['stage'] = t_match.apply(lambda row: get_stage(row), axis=1)
t_match


,Unnamed: 0,eid,PersonId_control,auid,year,Authorseq,last_author_seq,publication_type,doi,isAApub,PersonId,year_start1,year_end1,year_start2,year_end2,MoveType,stage
0,0,85080024884,30039,9279801600,2020,3,5,ar,10.1016/j.bushor.2020.01.009,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,1.0
11,2,84900854898,30039,9279801600,2014,3,3,ar,10.1108/jwl-08-2013-0057,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,0.0
22,4,84925100394,30039,7006724172,2015,2,2,ar,10.1080/0965254x.2014.914067,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,1.0
33,5,82555197073,30039,7006724172,2011,2,4,ar,10.1108/08858621211188948,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,0.0
44,8,79957896997,30039,7006724172,2011,4,4,ar,10.1108/10610421111134923,1.0,24853,1999,2014,2015.0,2020.0,M-PWI-HBCU,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468798,11878,85044068680,627531,8374772200,2014,1,3,ar,10.1080/10437797.2014.947899,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0
468820,11880,77951229585,627531,8374772200,2010,3,5,ar,10.1080/00377310903504940,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0
468842,11887,84955258941,627531,8374772200,2016,2,3,ar,10.1080/10437797.2016.1119016,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0
468864,11905,84929879065,627531,8374772200,2015,1,3,ar,10.1080/15228835.2015.1022683,1.0,620868,2012,2018,2018.0,2020.0,M-HBCU-PWI,0.0


In [ ]:
# concat tables
t_match['is_control'] = 1
t_match = t_match.drop(columns=['PersonId']).rename(columns={'PersonId_control': 'PersonId'})
t = pd.concat([t, t_match]).fillna({'is_control':0}).drop_duplicates()

t

,Unnamed: 0_x,eid,PersonId,auid,year,Authorseq,last_author_seq,publication_type,doi,isAApub,Unnamed: 0_y,univ1,year_start1,year_end1,isHBCU1,univ2,year_start2,year_end2,isHBCU2,MoveType,period1_len,period2_len,stage,Unnamed: 0,is_control
0,1.0,84857554607,886826,38860994900,2011,1,2,cp,10.1109/pimrc.2011.6139871,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
1,2.0,85083075356,886826,38860994900,2020,1,1,cp,10.1109/ccwc47524.2020.9031127,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
2,4.0,85035106301,886826,38860994900,2018,2,6,cp,10.1109/comst.2017.2773628,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
3,7.0,84883418788,886826,38860994900,2013,1,2,ar,10.1109/tsp.2013.2272554,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
4,10.0,85028758952,886826,38860994900,2017,4,4,ar,10.1109/tvt.2016.2619418,1.0,1563.0,North Carolina Agricultural and Technical Stat...,2018,2020,1.0,NaN,NaN,NaN,NaN,S,3.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468798,NaN,85044068680,627531,8374772200,2014,1,3,ar,10.1080/10437797.2014.947899,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11878.0,1.0
468820,NaN,77951229585,627531,8374772200,2010,3,5,ar,10.1080/00377310903504940,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11880.0,1.0
468842,NaN,84955258941,627531,8374772200,2016,2,3,ar,10.1080/10437797.2016.1119016,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11887.0,1.0
468864,NaN,84929879065,627531,8374772200,2015,1,3,ar,10.1080/15228835.2015.1022683,1.0,NaN,NaN,2012,2018,NaN,NaN,2018.0,2020.0,NaN,M-HBCU-PWI,NaN,NaN,0.0,11905.0,1.0


In [ ]:
tables1 = [df_au_eid,df_eid,
df_eid_citepct,
df_novel,
df_cd]

tables2 = [df_au_eid_match,df_eid_match,
df_eid_citepct_match,
df_novel_match,
df_cd_match]

for i in range(len(tables1)):
  table1 = tables1[i]
  table2 = tables2[i]
  table2 = table2.rename(columns={'PersonId_control': 'PersonId'})
  table1 = pd.concat([table1, table2]).drop_duplicates()
  tables1[i] = table1

df_au_eid, df_eid,df_eid_citepct,df_novel,df_cd = tables1[0], tables1[1], tables1[2], tables1[3], tables1[4]

# utilities

In [ ]:
# confidence interval
def calculate_confidence_interval(group):
    std = group.std()
    n = len(group)
    conf_level = 0.95
    dof = n - 1
    t_value = tstats.ppf((1 + conf_level) / 2, dof)
    se = std / np.sqrt(n)
    margin_error = t_value * se
    return margin_error


def winsorize_group(group):
    return winsorize(group, limits=[0, 0.05])

In [ ]:
import plotly.colors as pcolors

def generate_color_list(num_colors):
    # Get a list of colors from the Plotly color scales
    color_scale = pcolors.qualitative.Set1
    colors = color_scale * (num_colors // len(color_scale) + 1)

    color_list = []
    # Modify the colors to include the desired opacity
    for i in range(num_colors):
        color_list.append(f"rgba{colors[i][3:-1]}" + ", {})")
    return color_list

def generate_symbol_list(num_symbols):
    # Define a set of symbols to use
    symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'star', 'triangle-up', 'triangle-down']

    # Repeat the symbols to match the desired number
    symbol_list = symbols * (num_symbols // len(symbols) + 1)
    return symbol_list[:num_symbols]

# Example usage
num_items = 9
colors = generate_color_list(num_items)
symbols = generate_symbol_list(num_items)

# Print the generated lists
print(colors)
print(symbols)


['rgba(228,26,28, {})', 'rgba(55,126,184, {})', 'rgba(77,175,74, {})', 'rgba(152,78,163, {})', 'rgba(255,127,0, {})', 'rgba(255,255,51, {})', 'rgba(166,86,40, {})', 'rgba(247,129,191, {})', 'rgba(153,153,153, {})']
['circle', 'square', 'diamond', 'cross', 'x', 'star', 'triangle-up', 'triangle-down', 'circle']


# regression analysis

In [ ]:
pandas2ri.activate()

In [ ]:
def plot_styling(
    fig,
    size=(250, 250),
    title=None,
    xtitle="Year relative  to move",
    dtick=None,
):
    layout_params = dict(
      title=dict(text=title, x=0.5, xanchor='center', xref='container', font=dict(size=14)),
        width=size[1],
        height=size[0],
        template='simple_white',
        font=dict(size=11, family="Arial"),
        margin=dict(l=50, r=50, b=50, t=50),
    )

    fig.update_layout(
        **layout_params
    )

    fig.update_xaxes(title=dict(text=xtitle, standoff=0), dtick=dtick)

    return fig

In [ ]:
def add_lines_with_errorband(fig, x, y, lower, upper, name, color, text=None, colors=colors, showlegend=False, showband=True, dash=None, **kwargs):
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines+text",
            text=text,
            textposition='top left',
            name=name,
            line=dict(color=colors[color].format(1), width=1, dash=dash),
            showlegend=showlegend,
        ),**kwargs,
    )

    # add error band
    if showband:
        x_error = np.concatenate((x, x[::-1]))
        y_error = np.concatenate((lower, upper[::-1]))
        fig.add_trace(
            go.Scatter(
                x=x_error,
                y=y_error,
                fill="toself",
                fillcolor=colors[color].format(0.2),
                line=dict(color="rgba(255,255,255,0)"),
                showlegend=False,
            ),**kwargs,
        )

    return fig

In [ ]:
def add_points_with_errorbar(fig, x, y, lower, upper, name, color, colors=colors, showlegend=False,**kwargs,):
    array, arrayminus = upper - y, y - lower

    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="markers+lines",
            error_y=dict(type='data',
            symmetric=False,
            array=array,
            arrayminus=arrayminus),
            name=name,
            line=dict(color=colors[color].format(0.3), width=1),
            showlegend=showlegend,
        ),**kwargs,
    )

    return fig

In [ ]:
def reg_with_r(
    t,
    depvar,
    x,
    control,
    fix,
    family="ols",  # "poisson", "negbin", "logit", "gaussian"),
    **kwargs
):
    # convert to R dataframe
    r_dataframe = pandas2ri.py2rpy(t)

    if len(control) > 0:
        formula = f"{depvar} ~ {' + '.join(x)} + {' + '.join(control)} | {' + '.join(fix)}"
    else:
        formula = f"{depvar} ~ {' + '.join(x)} | {' + '.join(fix)}"

    print(formula)
    if family == "ols":
        model = fixest.feols(Formula(formula), data=r_dataframe, **kwargs)
    else:
        model = fixest.femlm(
            Formula(formula), data=r_dataframe, family=family, **kwargs
        )
    # Extract the coefficients, p-values, and confidence intervals
    # Create the R vector
    grades = robjects.FloatVector([0.01, 0.05, 0.1, 1])

    # Name the elements of the vector
    grades.names = robjects.StrVector([' A', ' B', ' C', ' D'])

    table = fixest.esttable(
        model, coefstat="confint", signif_code=grades
    )  #
    # convert to pandas
    pd_table = pandas2ri.rpy2py(table)
    pd_table.columns = ["var", "value"]
    return model, pd_table

In [ ]:
def time_interm_reg(
    df_tau_range,
    depvar,
    family="ols",  # "poisson", "negbin", "logit", "gaussian",
    xs=None,
    control=[],
    fix="PersonId year_asst Field_English".split(),
    **kwargs
):
    if not xs:
      # convert tau column to int
      df_tau_range["tau"] = df_tau_range["tau"].astype(int)

      # create dummy variables for each tau
      t = pd.concat(
          [df_tau_range, pd.get_dummies(df_tau_range["tau"], prefix="tau")], axis=1
      )
      t = t.drop(["tau_-5"], axis=1, errors="ignore")
      # replace - in column names with m
      t.columns = t.columns.str.replace("-", "m")
      tau_cols = [col for col in t.columns if col.startswith("tau_")]

      model, pd_table = reg_with_r(t, depvar=depvar, x=tau_cols, control=control, fix=fix, family=family, **kwargs) #, cluster='~PersonId'
    else:
      model, pd_table = reg_with_r(df_tau_range, depvar=depvar, x=xs, control=control, fix=fix, family=family,  **kwargs) #, cluster='~PersonId'

    pd_table = pd_table.query("var.str.contains('tau')").set_index("var")['value']
    pd_table = pd_table.str.extract(
        r"([-0-9.e]+)\s+([A-D])\s+\[([-0-9.e]+);\s+([-0-9.e]+)\]"
    ).applymap(lambda x: float(x) if x != 'A' and x != 'B' and x != 'C' and x != 'D' else x)
    pd_table.columns = ["beta", 'p', "lower", "upper"]
    return model, pd_table



In [ ]:
def get_df_tau_range(df, key_author=False):
    # pivot tau as columns and fill nan
    df_tau_range = (
        df.query("tau in @tau_range")
        # remove years and tau that don't have enough time to accumulate citations
        .query("year_asst + tau <= 2020").drop_duplicates()
    )
    if key_author:
        # first author or last author
        df_tau_range = df_tau_range.query(
            "Authorseq == 1 or Authorseq == last_author_seq"
        )
    return df_tau_range


# encode string column with numbers
def label_str_col(df, colname):
    df[colname] = pd.factorize(df[colname])[0].astype(np.uint16)
    return df

In [ ]:
# based table

df_base = (
    t[
        [
            "PersonId",
            "year_end1",
            "MoveType",
            'is_control',
        ]
    ]
    .rename(columns={"year_end1": 'year_asst'})
    .merge(
        df_au_eid[["PersonId", "eid", "year", "Authorseq", "last_author_seq"]],
        on=["PersonId"],
    )
    .merge(
        df_eid[
            [
                "PersonId",
                "eid",
                "srcid",
                "subfield_hybrid",
                "n_references",
                "avgnum_past_eid",
                "publication_type",
            ]
        ],
        on=["PersonId", "eid"],
    )
        .assign(tau=lambda t: t["year"] - t["year_asst"])
        # only key author
        #.query("Authorseq == 1 or Authorseq == last_author_seq")
    .drop_duplicates()
)
df_base['all'] = 1

tau_range = np.arange(-4, 5, 1)

In [ ]:
# stats
# of persons
print(df_base["PersonId"].nunique())
# of eids
print(df_base["eid"].nunique())

1816
43313


## productivity

In [ ]:
def fill_nan_with_zero(val, ind, col):
    if pd.isna(val):
        person_id = ind
        tau_range = t_range[t_range['PersonId'] == person_id]
        if not tau_range.empty:
            min_tau = tau_range['min_tau'].values[0]
            max_tau = tau_range['max_tau'].values[0]
            if min_tau <= col <= max_tau:
                return 0
    return val

In [ ]:
groupvar, group = 'is_control', [0, 1]
df_tau_ranges = []

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    t = df_base.query("MoveType	== @movetype")[[groupvar, "PersonId", "tau", "year_asst", "eid"]].drop_duplicates()
    t['all'] = 1

    # get min and max tau
    t['min_tau'] = t['year_asst'].apply(lambda x: max(tau_range[0], 2006-x))
    t['max_tau'] = t['year_asst'].apply(lambda x: min(tau_range[-1], 2020-x))
    t_range = t[['PersonId', 'min_tau', 'max_tau']].drop_duplicates()

    df_cite_norm = (
        t.groupby([groupvar, "PersonId", "tau"])
        .agg(mean1=("eid", "nunique"))
        .reset_index()
        .query("tau in @tau_range")
        .pivot(columns=['tau'], values=['mean1'], index=[groupvar, "PersonId"])
    )
    # fill 0 for tau range
    for ind, row in df_cite_norm.iterrows():
        for col in df_cite_norm.columns:
            val = row[col]
            row[col] = fill_nan_with_zero(val, ind[-1], col[-1])

    df_tau_range = (df_cite_norm
        .stack()
        .reset_index()
        .dropna()
        .rename(columns={"mean1": 'pub_num'})
    )

    df_tau_range['movetype'] = movetype
    df_tau_ranges.append(df_tau_range)

In [ ]:
# event study
groupvar, group = 'all', [1]
depvars = ["pub_num"]
xs = [f'i(tau, is_treat, ref=TRUE)'] # interaction term
control = []
fix=["PersonId"]

data = []

for m, movetype in enumerate(['M-HBCU-PWI', 'M-PWI-HBCU']):
  df_tau_range = df_tau_ranges[m]
  df_tau_range['all'] = 1
  for depvar in depvars:
    for domain in group:
      print(depvar, domain)
      t = df_tau_range.query(f"{groupvar} == @domain")
      t['is_treat'] = 1 - t['is_control']
      model, df = time_interm_reg(
          t,
          # use poisson
          family='poisson',
          depvar=depvar,
          xs=xs,
          control=control,
          fix=fix,
      )
      
      df['domain'] = domain
      df['movetype'] = movetype
      df['depvar'] = depvar
      data.append(df)

data = pd.concat(data)
data


pub_num 1
pub_num ~ i(tau, is_treat, ref=TRUE) | PersonId
pub_num 1
pub_num ~ i(tau, is_treat, ref=TRUE) | PersonId


,beta,p,lower,upper,domain,movetype,depvar
var,,,,,,,
is_treat x tau = -3,0.1907,C,-0.0359,0.4174,1,M-HBCU-PWI,pub_num
is_treat x tau = -2,0.0700,D,-0.1475,0.2874,1,M-HBCU-PWI,pub_num
is_treat x tau = -1,0.1479,D,-0.1684,0.4643,1,M-HBCU-PWI,pub_num
is_treat x tau = 0,0.1417,D,-0.1336,0.4169,1,M-HBCU-PWI,pub_num
is_treat x tau = 1,0.0912,D,-0.2417,0.4241,1,M-HBCU-PWI,pub_num
is_treat x tau = 2,0.0511,D,-0.2987,0.4010,1,M-HBCU-PWI,pub_num
is_treat x tau = 3,0.2786,D,-0.0835,0.6406,1,M-HBCU-PWI,pub_num
is_treat x tau = 4,0.1954,D,-0.1767,0.5676,1,M-HBCU-PWI,pub_num
is_treat x tau = -3,-0.1436,D,-0.4067,0.1195,1,M-PWI-HBCU,pub_num


In [ ]:
# plot
y_range = (data['lower'].min() - 0.1, data['upper'].max() + 0.1)
fig = go.Figure()

for m, movetype in enumerate(['M-HBCU-PWI', 'M-PWI-HBCU']):
    for j, d in enumerate(depvars):
        for i, domain in enumerate(group):

            t1_domain = data.query("domain == @domain and movetype==@movetype and depvar == @d")
            t1 = t1_domain.dropna()

            x = tau_range[1:]
            y1 = t1["beta"].to_numpy()
            y1_upper, y1_lower = (
                t1["upper"].to_numpy(),
                t1["lower"].to_numpy(),
            )
            text=t1['p'].map({"A": '***', 'B': '**', 'C': '*', "D": ""})
            # get color
            if movetype == 'M-HBCU-PWI':
                color = 1
            else:
                color = 4

            fig = add_lines_with_errorband(
                fig, x, y1, text=text, upper=y1_upper, lower=y1_lower, name=domain, color=color, showlegend=False#, dash=dashes[j]
            )

            fig = plot_styling(fig, size=(250, 250), title=None)
            fig.update_xaxes(range=(tau_range[1], tau_range[-1]), mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
            fig.update_yaxes(
                title=dict(text='Diff. in pubs. from control', standoff=0),
                titlefont=dict(color="black"),
                tickfont=dict(color="black"),
                range=y_range,
                mirror=True,
                zeroline=True,
                zerolinecolor="rgba(128,128,128, 0.3)",
            )

            fig.show()


# citations

In [ ]:
# get their tau and cite
cite_metric, jif_metric, novel_metric, cd_metric = 'cite3_pct', 'jif_pct', "novelty_pct", "cd5_pct"
depvars = [cite_metric, jif_metric, novel_metric, cd_metric]

t_base = (
    df_base.merge(
        df_eid_citepct[["eid", cite_metric]],
        on=["eid"],
        how='left',
    )
    .merge(df_jif[['srcid', 'year', jif_metric]], on=['srcid', "year"], how='left')
    .merge(df_novel[["eid", novel_metric]], on=["eid"], how="left")
    .merge(df_cd[["eid", cd_metric]], on=["eid"], how="left")
    .fillna(
        {
            "n_references": 0,
            "avgnum_past_eid": 0,
        }
    )
)

# remove 2006 cd5
t_base.loc[t_base['year'] <= 2006, 'cd5_pct'] = np.nan

# # use binary, threshold = 90
# t_base['cite3_pct'] = t_base['cite3_pct'].apply(lambda x: 1 if x<=0.1 else 0) #>=0.9
# t_base['jif_pct'] = t_base['jif_pct'].apply(lambda x: 1 if x<=0.1 else 0)
# t_base['novelty_pct'] = t_base['novelty_pct'].apply(lambda x: 1 if x>=0.75 else 0)
# t_base['cd5_pct'] = t_base['cd5_pct'].apply(lambda x: 1 if x>=0.75 else 0)

In [ ]:
# event study

# break into taus
groupvar, group = 'all', [1]
depvars = [cite_metric, jif_metric] #, novel_metric, cd_metric
xs = [f'i(tau, is_treat, ref=TRUE)'] # interaction term
control = "avgnum_past_eid".split()
fix = "subfield_hybrid publication_type".split() # 

data = []

for m, movetype in enumerate(['M-HBCU-PWI', 'M-PWI-HBCU']):
    for method in ["all"]: #, "max", "min"
        for depvar in depvars:

            t = (
                get_df_tau_range(t_base).query('MoveType == @movetype')[["eid", depvar, "tau", 'is_control', groupvar] + control + fix]
                .dropna()
                .drop_duplicates()
            )
            t['is_treat'] = 1 - t['is_control']
            df_tau_range = t

            for domain in group:
                print(depvar, domain)
                t = df_tau_range.query(f"{groupvar} == @domain")

                model, df = time_interm_reg(
                    t,
                    depvar=depvar,
                    family='ols',
                    xs=xs,
                    control=control,
                    fix=fix                )

                df["domain"] = domain
                df["method"] = method
                df["depvar"] = depvar
                df['movetype'] = movetype
                data.append(df)

data = pd.concat(data)
data

cite3_pct 1
cite3_pct ~ i(tau, is_treat, ref=TRUE) + avgnum_past_eid | subfield_hybrid + publication_type
jif_pct 1
jif_pct ~ i(tau, is_treat, ref=TRUE) + avgnum_past_eid | subfield_hybrid + publication_type
cite3_pct 1
cite3_pct ~ i(tau, is_treat, ref=TRUE) + avgnum_past_eid | subfield_hybrid + publication_type
jif_pct 1
jif_pct ~ i(tau, is_treat, ref=TRUE) + avgnum_past_eid | subfield_hybrid + publication_type


,beta,p,lower,upper,domain,method,depvar,movetype
var,,,,,,,,
is_treat x tau = -3,-0.0083,D,-0.0754,0.0587,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = -2,0.0277,D,-0.0511,0.1065,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = -1,0.0100,D,-0.0460,0.0660,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = 0,0.0255,D,-0.0458,0.0968,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = 1,0.0230,D,-0.0569,0.1028,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = 2,0.0223,D,-0.0510,0.0956,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = 3,0.0159,D,-0.0459,0.0776,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = 4,-0.0215,D,-0.0957,0.0527,1,all,cite3_pct,M-HBCU-PWI
is_treat x tau = -3,-0.0280,D,-0.1062,0.0501,1,all,jif_pct,M-HBCU-PWI


In [ ]:
# high cited/JIF pub number

groupvar, group = 'is_control', [0, 1]
df_tau_ranges = []
intervals = ['cite3_pct >= 0.9', 'jif_pct >= 0.9']

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    t = t_base.query(f"MoveType	== @movetype and {intervals[0]}")[[groupvar, "PersonId", "tau", "year_asst", "eid"]].drop_duplicates() # 
    t['all'] = 1

    # get min and max tau
    t['min_tau'] = t['year_asst'].apply(lambda x: max(tau_range[0], 2006-x))
    t['max_tau'] = t['year_asst'].apply(lambda x: min(tau_range[-1], 2020-x))
    t_range = t[['PersonId', 'min_tau', 'max_tau']].drop_duplicates()

    df_cite_norm = (
        t.groupby([groupvar, "PersonId", "tau"])
        .agg(mean1=("eid", 'nunique'))
        .reset_index()
        .query("tau in @tau_range")
        .pivot(columns=['tau'], values=['mean1'], index=[groupvar, "PersonId"])
    )
    # fill 0 for tau range
    for ind, row in df_cite_norm.iterrows():
        for col in df_cite_norm.columns:
            val = row[col]
            row[col] = fill_nan_with_zero(val, ind[-1], col[-1])

    df_tau_range = (df_cite_norm
        .stack()
        .reset_index()
        .dropna()
        .rename(columns={"mean1": 'pub_num'})
    )

    df_tau_range['movetype'] = movetype
    df_tau_ranges.append(df_tau_range)


# event study
groupvar, group = 'all', [1]
depvars = ["pub_num"]
xs = [f'i(tau, is_treat, ref=TRUE)'] # interaction term
control = []
fix=["PersonId"]

data = []

for m, movetype in enumerate(['M-HBCU-PWI', 'M-PWI-HBCU']):
    df_tau_range = df_tau_ranges[m]
    df_tau_range['all'] = 1
    for depvar in depvars:
        for domain in group:
            print(depvar, domain)
            t = df_tau_range.query(f"{groupvar} == @domain")
            t['is_treat'] = 1 - t['is_control']
            model, df = time_interm_reg(
                t,
                # use poisson
                family='poisson',
                depvar=depvar,
                xs=xs,
                control=control,
                fix=fix,
            )
            
            df['domain'] = domain
            df['movetype'] = movetype
            df['depvar'] = depvar
            data.append(df)

data = pd.concat(data)
data


pub_num 1
pub_num ~ i(tau, is_treat, ref=TRUE) | PersonId
pub_num 1
pub_num ~ i(tau, is_treat, ref=TRUE) | PersonId


,beta,p,lower,upper,domain,movetype,depvar
var,,,,,,,
is_treat x tau = -3,0.3795,D,-0.1643,0.9233,1,M-HBCU-PWI,pub_num
is_treat x tau = -2,0.3795,D,-0.1408,0.8998,1,M-HBCU-PWI,pub_num
is_treat x tau = -1,0.6539,B,0.0942,1.2140,1,M-HBCU-PWI,pub_num
is_treat x tau = 0,0.5705,D,-0.1846,1.3260,1,M-HBCU-PWI,pub_num
is_treat x tau = 1,0.6743,D,-0.3352,1.6840,1,M-HBCU-PWI,pub_num
is_treat x tau = 2,0.9377,C,-0.0647,1.9400,1,M-HBCU-PWI,pub_num
is_treat x tau = 3,1.0660,B,0.2260,1.9060,1,M-HBCU-PWI,pub_num
is_treat x tau = 4,0.6842,B,0.0491,1.3190,1,M-HBCU-PWI,pub_num
is_treat x tau = -3,-0.0408,D,-0.4587,0.3771,1,M-PWI-HBCU,pub_num


In [ ]:
# plot
y_range = (data['lower'].min() - 0.1, data['upper'].max() + 0.1)
fig = go.Figure()

for m, movetype in enumerate(['M-HBCU-PWI', 'M-PWI-HBCU']):

    for j, d in enumerate(depvars):
        for i, domain in enumerate(group):

            t1_domain = data.query("domain == @domain and movetype==@movetype and depvar == @d")
            t1 = t1_domain.dropna()

            x = tau_range[1:]
            y1 = t1["beta"].to_numpy()
            y1_upper, y1_lower = (
                t1["upper"].to_numpy(),
                t1["lower"].to_numpy(),
            )
            text=t1['p'].map({"A": '***', 'B': '**', 'C': '*', "D": ""})

            # get color
            if movetype == 'M-HBCU-PWI':
                color = 1
            else:
                color = 4

            fig = add_lines_with_errorband(
                fig, x, y1, text=text, upper=y1_upper, lower=y1_lower, name=domain, color=color, showlegend=False#, dash=dashes[j]
            )

        fig = plot_styling(fig, size=(250, 250), title=None)
        fig.update_xaxes(range=(tau_range[1], tau_range[-1]), mirror=True, zeroline=True, zerolinecolor="rgba(128,128,128, 0.3)")
        fig.update_yaxes(
            title=dict(text='Diff. in top-citation pubs<br>from control', standoff=0),
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            range=y_range,
            mirror=True,
            zeroline=True,
            zerolinecolor="rgba(128,128,128, 0.3)",
        )

        fig.show()


# collaboration

* number of authors
* fresh collaborators
* is cross-college
* is international
* seniority

In [ ]:
df_fresh_collab = pd.read_csv(f'path/hbcu/hbcu_collab_fresh.csv')
df_collab_af = pd.read_csv(f'path/hbcu/hbcu_collab_afnum.csv')
df_senior = pd.read_csv(f'path/hbcu/hbcu_collab_senior.csv')


In [ ]:
# convert to binary
df_collab_af['is_cross_inst'] = df_collab_af['af_num'].apply(lambda x: 1 if x > 1 else 0)
df_collab_af['is_international'] = df_collab_af['ctry_num'].apply(lambda x: 1 if x > 1 else 0)

# get junior percentage
df_senior['is_junior'] = df_senior['seniority'].apply(lambda x: 1 if x < 7 else 0)
numer = df_senior.query("is_junior == 1").groupby('eid').agg(numer=('auid', 'nunique')).reset_index()
denom = df_senior.groupby('eid').agg(denom=('auid', 'nunique')).reset_index()
df_senior = denom.merge(numer, on=['eid'], how='left').fillna(0)
df_senior['junior_pct'] = df_senior['numer'] / df_senior['denom']
df_senior

,eid,denom,numer,junior_pct
0,27444438189,4,3.0,0.750000
1,27544445275,8,3.0,0.375000
2,27844457541,3,2.0,0.666667
3,28244435776,2,1.0,0.500000
4,28444476217,3,2.0,0.666667
...,...,...,...,...
32157,85172021658,10,2.0,0.200000
32158,85172022223,3,1.0,0.333333
32159,85172022652,2,2.0,1.000000
32160,85172033503,6,4.0,0.666667


In [ ]:
# prepare base
df_base_collab = (
    df_base.merge(df_fresh_collab[['PersonId', "eid", "fresh_perc"]], on=['PersonId', "eid"], how="left")
    .merge(df_collab_af[["eid", 'is_cross_inst', 'is_international']], on=["eid"], how="left")
    .merge(df_senior[["eid", 'junior_pct']], on=["eid"], how="left")
    .fillna(
        {
            "n_references": 0,
            'fresh_perc': 0,
            "avgnum_past_eid": 0,
        }
    )
    .drop_duplicates()
)

In [ ]:
# break into taus
groupvar, group = 'all', [1]

# get their tau and cite
cite_metric, jif_metric, novel_metric, cd_metric = 'cite3_pct', 'jif_pct', "novelty_pct", "cd5_pct"

t_base = (
    df_base_collab.merge(
        df_eid_citepct[["eid", cite_metric]],
        on=["eid"],
        how='left',
    )
    .merge(df_jif[['srcid', 'year', jif_metric]], on=['srcid', "year"], how='left')
    .merge(df_novel[["eid", novel_metric]], on=["eid"], how="left")
    .merge(df_cd[["eid", cd_metric]], on=["eid"], how="left")
    .fillna(
        {
            "n_references": 0,
            "avgnum_past_eid": 0,
        }
    )
    .drop_duplicates()
)

# remove 2006 cd5
t_base.loc[t_base['year'] <= 2006, 'cd5_pct'] = np.nan

t_base['all'] = 1

In [ ]:
# use collab as dependent variable

# break into taus
groupvar, group = 'all', [1]
depvars = "last_author_seq fresh_perc is_cross_inst is_international junior_pct".split()
control = []
fix = "PersonId subfield_hybrid publication_type".split()

data = []

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    for method in ["all", "max", "min"]:
        for depvar in depvars:

            t = (
                get_df_tau_range(t_base).query("MoveType == @movetype")[["eid", depvar, "tau", groupvar] + control + fix]
                .dropna()
                .drop_duplicates()
            )

            if method == "max":
                # top quartile
                t = t[
                    t[depvar]
                    >= t.groupby([groupvar, "PersonId", "tau"])[depvar].transform("max")
                ]

            if method == "min":
                # bottom quartile
                t = t[
                    t[depvar]
                    <= t.groupby([groupvar, "PersonId", "tau"])[depvar].transform("min")
                ]

            df_tau_range = t

            for domain in group:
                print(depvar, domain)
                t = df_tau_range.query(f"{groupvar} == @domain")

                model, df = time_interm_reg(
                    t,
                    depvar=depvar,
                    control=control,
                    fix=fix,
                )

                df["domain"] = domain
                df["method"] = method
                df["depvar"] = depvar
                df['movetype'] = movetype
                data.append(df)

data = pd.concat(data)
data

last_author_seq 1
last_author_seq ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId + subfield_hybrid + publication_type
fresh_perc 1
fresh_perc ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId + subfield_hybrid + publication_type
is_cross_inst 1
is_cross_inst ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId + subfield_hybrid + publication_type
is_international 1
is_international ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId + subfield_hybrid + publication_type
junior_pct 1
junior_pct ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId + subfield_hybrid + publication_type
last_author_seq 1
last_author_seq ~ tau_m4 + tau_m3 + tau_m2 + tau_m1 + tau_0 + tau_1 + tau_2 + tau_3 + tau_4 + tau_5 | PersonId + subfield_hybrid + publication_type
fresh_perc 1
fresh_perc ~ 

,beta,p,lower,upper,domain,method,depvar,movetype
var,,,,,,,,
tau_m4,0.4324,D,-0.3033,1.1680,1,all,last_author_seq,M-HBCU-PWI
tau_m3,0.2806,D,-0.5399,1.1010,1,all,last_author_seq,M-HBCU-PWI
tau_m2,1.1220,C,-0.1346,2.3790,1,all,last_author_seq,M-HBCU-PWI
tau_m1,0.6601,D,-0.1446,1.4650,1,all,last_author_seq,M-HBCU-PWI
tau_0,1.3150,C,-0.0218,2.6510,1,all,last_author_seq,M-HBCU-PWI
...,...,...,...,...,...,...,...,...
tau_1,-0.0198,D,-0.1850,0.1453,1,min,junior_pct,M-PWI-HBCU
tau_2,-0.0384,D,-0.2593,0.1826,1,min,junior_pct,M-PWI-HBCU
tau_3,-0.0531,D,-0.2337,0.1275,1,min,junior_pct,M-PWI-HBCU


In [ ]:
# plot

depvars = "last_author_seq fresh_perc is_cross_inst is_international junior_pct".split()

for movetype in ['M-HBCU-PWI', 'M-PWI-HBCU']:
    fig = make_subplots(rows=3, cols=5, horizontal_spacing=0.1)

    for d, depvar in enumerate(depvars):
        for k, method in enumerate(["all", "max", "min"]):
            for i, domain in enumerate(group):
                t1_domain = data.query("domain == @domain and movetype == @movetype and depvar == @depvar and method == @method")
                t1 = t1_domain.dropna()

                x = tau_range[1:]
                y1 = t1["beta"].to_numpy()
                y1_upper, y1_lower = (
                    t1["upper"].to_numpy(),
                    t1["lower"].to_numpy(),
                )

                fig = add_lines_with_errorband(
                    fig, x, y1, upper=y1_upper, lower=y1_lower, name=domain, color=i, showlegend=False, row=k+1, col=d+1
                )

                # mark 0-1 zone
                fig.add_vrect(x0=0, x1=1, line_width=0, fillcolor="gray", opacity=0.2, row=k+1, col=d+1)

    fig = plot_styling(fig, size=(600, 900), title=None)
    fig.update_yaxes(
        title=dict(text='Coef.', standoff=0),
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
    )

    fig.show()
